In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, Subset

# Define transform to preprocess the data
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Normalize images
])

# Load the SVHN dataset
svhn_train = datasets.SVHN(root='./data', split='train', download=True, transform=transform)

# Create a subset of the dataset (25%)
subset_indices = torch.randperm(len(svhn_train))[:len(svhn_train)//4]
svhn_subset = Subset(svhn_train, subset_indices)

# Define the dataloaders
train_loader = DataLoader(svhn_subset, batch_size=32, shuffle=True)

# Define the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define model architectures
models_dict = {
    'LeNet-5': models.LeNet,
    'AlexNet': models.alexnet,
    'VGG': models.vgg16,
    'ResNet-18': models.resnet18,
    'ResNet-50': models.resnet50,
    'ResNet-101': models.resnet101
}

# Train and evaluate each model
for model_name, model_func in models_dict.items():
    print(f"Training {model_name}...")

    # Load pretrained model
    model = model_func(pretrained=True)

    # Modify the last layer to fit SVHN output size
    if 'resnet' in model_name:
        num_features = model.fc.in_features
        model.fc = nn.Linear(num_features, 10)
    else:
        num_features = model.classifier[-1].in_features
        model.classifier[-1] = nn.Linear(num_features, 10)

    # Move model to device
    model.to(device)

    # Define loss function and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

    # Train the model
    for epoch in range(5):  # Train for 5 epochs for demonstration
        running_loss = 0.0
        for i, data in enumerate(train_loader, 0):
            inputs, labels = data[0].to(device), data[1].to(device)

            optimizer.zero_grad()

            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            if i % 100 == 99:
                print(f"[{epoch + 1}, {i + 1}] loss: {running_loss / 100:.3f}")
                running_loss = 0.0

    print(f"Finished training {model_name}")

    # Evaluate the model
    correct = 0
    total = 0
    with torch.no_grad():
        for data in train_loader:
            images, labels = data[0].to(device), data[1].to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(f"Accuracy of the network on the {len(svhn_subset)} training images: {100 * correct / total:.2f}%")


Using downloaded and verified file: ./data/train_32x32.mat


AttributeError: module 'torchvision.models' has no attribute 'LeNet'

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, Subset

# Define LeNet-5 architecture
class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, kernel_size=5)
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(6, 16, kernel_size=5)
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool1(torch.relu(self.conv1(x)))
        x = self.pool2(torch.relu(self.conv2(x)))
        x = torch.flatten(x, 1)  # flatten all dimensions except batch
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Define transform to preprocess the data
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Normalize images
])

# Load the SVHN dataset
svhn_train = datasets.SVHN(root='./data', split='train', download=True, transform=transform)

# Create a subset of the dataset (25%)
subset_indices = torch.randperm(len(svhn_train))[:len(svhn_train)//4]
svhn_subset = Subset(svhn_train, subset_indices)

# Define the dataloaders
train_loader = DataLoader(svhn_subset, batch_size=32, shuffle=True)

# Define the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define model architectures
models_dict = {
    'LeNet-5': LeNet,
    'AlexNet': models.alexnet,
    'VGG': models.vgg16,
    'ResNet-18': models.resnet18,
    'ResNet-50': models.resnet50,
    'ResNet-101': models.resnet101
}

# Train and evaluate each model
for model_name, model_func in models_dict.items():
    print(f"Training {model_name}...")

    # Load pretrained model
    if model_name == 'LeNet-5':
        model = model_func()
    else:
        model = model_func(pretrained=True)

    # Modify the last layer to fit SVHN output size
    if 'resnet' in model_name:
        num_features = model.fc.in_features
        model.fc = nn.Linear(num_features, 10)
    else:
        num_features = model.classifier[-1].in_features
        model.classifier[-1] = nn.Linear(num_features, 10)

    # Move model to device
    model.to(device)

    # Define loss function and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

    # Train the model
    for epoch in range(5):  # Train for 5 epochs for demonstration
        running_loss = 0.0
        for i, data in enumerate(train_loader, 0):
            inputs, labels = data[0].to(device), data[1].to(device)

            optimizer.zero_grad()

            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            if i % 100 == 99:
                print(f"[{epoch + 1}, {i + 1}] loss: {running_loss / 100:.3f}")
                running_loss = 0.0

    print(f"Finished training {model_name}")

    # Evaluate the model
    correct = 0
    total = 0
    with torch.no_grad():
        for data in train_loader:
            images, labels = data[0].to(device), data[1].to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(f"Accuracy of the network on the {len(svhn_subset)} training images: {100 * correct / total:.2f}%")


100%|██████████| 182040794/182040794 [00:04<00:00, 36702370.77it/s]


Training LeNet-5...


AttributeError: 'LeNet' object has no attribute 'classifier'

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, Subset

# Define LeNet-5 architecture
class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, kernel_size=5)
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(6, 16, kernel_size=5)
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool1(torch.relu(self.conv1(x)))
        x = self.pool2(torch.relu(self.conv2(x)))
        x = torch.flatten(x, 1)  # flatten all dimensions except batch
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Define transform to preprocess the data
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Normalize images
])

# Load the SVHN dataset
svhn_train = datasets.SVHN(root='./data', split='train', download=True, transform=transform)

# Create a subset of the dataset (25%)
subset_indices = torch.randperm(len(svhn_train))[:len(svhn_train)//4]
svhn_subset = Subset(svhn_train, subset_indices)

# Define the dataloaders
train_loader = DataLoader(svhn_subset, batch_size=32, shuffle=True)

# Define the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define model architectures
models_dict = {
    'LeNet-5': LeNet,
    'AlexNet': models.alexnet,
    'VGG': models.vgg16,
    'ResNet-18': models.resnet18,
    'ResNet-50': models.resnet50,
    'ResNet-101': models.resnet101
}

# Train and evaluate each model
for model_name, model_func in models_dict.items():
    print(f"Training {model_name}...")

    # Load pretrained model
    if model_name == 'LeNet-5':
        model = model_func()
    else:
        model = model_func(pretrained=True)
        if 'resnet' in model_name:
            num_features = model.fc.in_features
            model.fc = nn.Linear(num_features, 10)
        else:
            num_features = model.classifier[6].in_features
            model.classifier[6] = nn.Linear(num_features, 10)

    # Move model to device
    model.to(device)

    # Define loss function and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

    # Train the model
    for epoch in range(5):  # Train for 5 epochs for demonstration
        running_loss = 0.0
        for i, data in enumerate(train_loader, 0):
            inputs, labels = data[0].to(device), data[1].to(device)

            optimizer.zero_grad()

            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            if i % 100 == 99:
                print(f"[{epoch + 1}, {i + 1}] loss: {running_loss / 100:.3f}")
                running_loss = 0.0

    print(f"Finished training {model_name}")

    # Evaluate the model
    correct = 0
    total = 0
    with torch.no_grad():
        for data in train_loader:
            images, labels = data[0].to(device), data[1].to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(f"Accuracy of the network on the {len(svhn_subset)} training images: {100 * correct / total:.2f}%")


Using downloaded and verified file: ./data/train_32x32.mat
Training LeNet-5...
[1, 100] loss: 2.298
[1, 200] loss: 2.285
[1, 300] loss: 2.271
[1, 400] loss: 2.260
[1, 500] loss: 2.246
[2, 100] loss: 2.236
[2, 200] loss: 2.243
[2, 300] loss: 2.254
[2, 400] loss: 2.240
[2, 500] loss: 2.237
[3, 100] loss: 2.230
[3, 200] loss: 2.246
[3, 300] loss: 2.237
[3, 400] loss: 2.242
[3, 500] loss: 2.244
[4, 100] loss: 2.240
[4, 200] loss: 2.243
[4, 300] loss: 2.241
[4, 400] loss: 2.237
[4, 500] loss: 2.232
[5, 100] loss: 2.233
[5, 200] loss: 2.229
[5, 300] loss: 2.238
[5, 400] loss: 2.229
[5, 500] loss: 2.246
Finished training LeNet-5
Accuracy of the network on the 18314 training images: 18.81%
Training AlexNet...


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/alexnet-owt-7be5be79.pth" to /root/.cache/torch/hub/checkpoints/alexnet-owt-7be5be79.pth
100%|██████████| 233M/233M [00:05<00:00, 47.7MB/s]


RuntimeError: Given input size: (256x1x1). Calculated output size: (256x0x0). Output size is too small

In [4]:
import torch
import torchvision.transforms as transforms
from torchvision.datasets import SVHN
from torch.utils.data import DataLoader
from torchvision import models
import torch.nn as nn
import torch.optim as optim

# Define transforms for the dataset
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Normalize to [-1, 1]
])

# Load the SVHN dataset
train_dataset = SVHN(root='./data', split='train', download=True, transform=transform)
test_dataset = SVHN(root='./data', split='test', download=True, transform=transform)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# Define device (GPU if available, else CPU)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


Using downloaded and verified file: ./data/train_32x32.mat


100%|██████████| 64275384/64275384 [00:01<00:00, 45340418.34it/s]


In [ ]:
def train_model(model, criterion, optimizer, num_epochs=5):
    model.to(device)
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * inputs.size(0)
        epoch_loss = running_loss / len(train_loader.dataset)
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}")

# Function to evaluate the model
def evaluate_model(model):
    model.to(device)
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = correct / total
    print(f"Test Accuracy: {accuracy:.4f}")

# Load pretrained ResNet models
resnet18 = models.resnet18(pretrained=True)
resnet50 = models.resnet50(pretrained=True)
resnet101 = models.resnet101(pretrained=True)

# Modify the last fully connected layer for our classification task
num_classes = 10
resnet18.fc = nn.Linear(resnet18.fc.in_features, num_classes)
resnet50.fc = nn.Linear(resnet50.fc.in_features, num_classes)
resnet101.fc = nn.Linear(resnet101.fc.in_features, num_classes)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer18 = optim.SGD(resnet18.parameters(), lr=0.001, momentum=0.9)
optimizer50 = optim.SGD(resnet50.parameters(), lr=0.001, momentum=0.9)
optimizer101 = optim.SGD(resnet101.parameters(), lr=0.001, momentum=0.9)

# Train and evaluate the models
print("Training ResNet-18...")
train_model(resnet18, criterion, optimizer18)
evaluate_model(resnet18)

print("Training ResNet-50...")
train_model(resnet50, criterion, optimizer50)
evaluate_model(resnet50)

print("Training ResNet-101...")
train_model(resnet101, criterion, optimizer101)
evaluate_model(resnet101)


Training ResNet-18...
Epoch [1/5], Loss: 0.7970
Epoch [2/5], Loss: 0.3481
Epoch [3/5], Loss: 0.2590
Epoch [4/5], Loss: 0.2109
Epoch [5/5], Loss: 0.1717
Test Accuracy: 0.9196
Training ResNet-50...
Epoch [1/5], Loss: 0.7098
Epoch [2/5], Loss: 0.2753
Epoch [3/5], Loss: 0.1915
Epoch [4/5], Loss: 0.1398
Epoch [5/5], Loss: 0.1071
